In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create network
model = Network('model')

# Block 1: nodes
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)
node3 = Queue(model, 'Queue2', SchedStrategy.PS)
node3.set_number_of_servers(2)  # Queue2 has 2 servers

In [3]:
# Block 2: classes
N = [1, 0, 3, 0]  # Population for each class
jobclass1 = ClosedClass(model, 'Class1', N[0], node1, 0)
jobclass2 = ClosedClass(model, 'Class2', N[1], node1, 0)
jobclass3 = ClosedClass(model, 'Class3', N[2], node1, 0)
jobclass4 = ClosedClass(model, 'Class4', N[3], node1, 0)

In [4]:
# Set service times for all stations
# Delay
node1.set_service(jobclass1, Exp.fit_mean(1.0))
node1.set_service(jobclass2, Exp.fit_mean(1.0/2.0))
node1.set_service(jobclass3, Exp.fit_mean(1.0))
node1.set_service(jobclass4, Exp.fit_mean(1.0))

# Queue1
node2.set_service(jobclass1, Exp.fit_mean(1.0/3.0))
node2.set_service(jobclass2, Exp.fit_mean(1.0/4.0))
node2.set_service(jobclass3, Exp.fit_mean(1.0/5.0))
node2.set_service(jobclass4, Exp.fit_mean(1.0))

# Queue2
node3.set_service(jobclass1, Exp.fit_mean(1.0))
node3.set_service(jobclass2, Exp.fit_mean(1.0/3.0))
node3.set_service(jobclass3, Exp.fit_mean(1.0/5.0))
node3.set_service(jobclass4, Exp.fit_mean(1.0/2.0))

In [5]:
# Block 3: routing with class switching
RT = model.init_routing_matrix()

# Set up the routing matrix with class switching
nodes = [node1, node2, node3]
jobclasses = [jobclass1, jobclass2, jobclass3, jobclass4]

# Class1 routing: Delay -> Queue1 -> Queue2 -> switch to Class2
RT.set(jobclass1, jobclass1, node1, node2, 1.0)  # Delay -> Queue1
RT.set(jobclass1, jobclass1, node2, node3, 1.0)  # Queue1 -> Queue2
RT.set(jobclass1, jobclass2, node3, node1, 1.0)  # Queue2 -> Delay (switch to Class2)

# Class2 routing: Delay -> Queue1 -> Queue2 -> switch to Class1
RT.set(jobclass2, jobclass2, node1, node2, 1.0)  # Delay -> Queue1
RT.set(jobclass2, jobclass2, node2, node3, 1.0)  # Queue1 -> Queue2
RT.set(jobclass2, jobclass1, node3, node1, 1.0)  # Queue2 -> Delay (switch to Class1)

# Class3 routing: Delay -> Queue1 -> Queue2 -> switch to Class4
RT.set(jobclass3, jobclass3, node1, node2, 1.0)  # Delay -> Queue1
RT.set(jobclass3, jobclass3, node2, node3, 1.0)  # Queue1 -> Queue2
RT.set(jobclass3, jobclass4, node3, node1, 1.0)  # Queue2 -> Delay (switch to Class4)

# Class4 routing: Delay -> Queue2 -> switch to Class3
RT.set(jobclass4, jobclass4, node1, node3, 1.0)  # Delay -> Queue2
RT.set(jobclass4, jobclass3, node3, node1, 1.0)  # Queue2 -> Delay (switch to Class3)

model.link(RT)

In [6]:
# Set initial state: all jobs at Queue2
n = np.array([[0,0,0,0],        # Delay: no jobs
              [0,0,0,0],        # Queue1: no jobs
              [N[0],N[1],N[2],N[3]]])  # Queue2: all jobs

nodes = [node1, node2, node3]
for i in range(len(nodes)):
    nodes[i].set_state(n[i])

In [7]:
# Solver options
options = Solver.defaultOptions
options.verbose = 1

In [8]:
# Solve with CTMC
solver_ctmc = CTMC(model, options)
Pr_ctmc = solver_ctmc.prob_sys_aggr()

CTMC getProbSysAggr failed: java.lang.IllegalArgumentException: Outside of matrix bounds


In [9]:
# Solve with NC (exact method)
options_nc = Solver.defaultOptions
options_nc.verbose = 1
options_nc.method = 'exact'

solver_nc = NC(model, options_nc)
Pr_nc = solver_nc.prob_sys_aggr()

In [10]:
# Solve with JMT (simulation)
solver_jmt = JMT(model, samples=10000, seed=532733)
Pr_jmt = solver_jmt.prob_sys_aggr()

Jul 29, 2025 12:13:54 PM jline.io.InputOutputKt line_printf
SEVERE: [LINE2JSIMG] Error during LINE to JSIMG conversion: Unable to determine root folder.
Error in sampleSysAggr: Error during network copy operation: [LINE2JSIMG] Error during LINE to JSIMG conversion: Unable to determine root folder.
